<a href="https://colab.research.google.com/github/JatinK169/Amazon_ML_Challenge/blob/main/AmazonMLChallengeTheAIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Vyoum's drive:
train_path = 'drive/My Drive/datasetb2d9982/dataset/train.csv'
train_data = pd.read_csv(train_path)
train_data.describe()

,PRODUCT_ID,PRODUCT_TYPE_ID,PRODUCT_LENGTH
count,2.249698e+06,2.249698e+06,2.249698e+06
mean,1.499795e+06,4.000456e+03,4.071839e+03
std,8.661944e+05,3.966146e+03,1.351685e+06
min,1.000000e+00,0.000000e+00,1.000000e+00
25%,7.494795e+05,2.300000e+02,5.118110e+02
50%,1.499558e+06,2.916000e+03,6.630000e+02
75%,2.250664e+06,6.403000e+03,1.062992e+03
max,2.999999e+06,1.342000e+04,1.885801e+09


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('drive/My Drive/datasetb2d9982/dataset/train.csv', index_col='PRODUCT_ID')
X_test_full = pd.read_csv('drive/My Drive/datasetb2d9982/dataset/test.csv', index_col='PRODUCT_ID')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['PRODUCT_LENGTH'], inplace=True)
y = X.PRODUCT_LENGTH
X.drop(['PRODUCT_LENGTH'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [ ]:
X_test.head()

,PRODUCT_TYPE_ID
PRODUCT_ID,
604373,6142
1729783,1622
1871949,7540
1107571,12442
624253,6318


In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error


## Define the model
## XGBoost is a powerful library for building and training decision trees
'''XGBoost also provides a regression implementation called XGRegressor,
which is used for building and training regression models. XGRegressor uses
gradient boosting algorithms to iteratively improve the accuracy of the
regression model.'''
my_model_2 = XGBRegressor(n_estimators = 100000, learning_rate = 0.09, n_jobs = 6)
'''
learning_rate: It controls the step size at each iteration while moving toward
a minimum of a loss function.
n_estimators: It is the number of trees to be created in the model.
njobs: It is used to specify the number of parallel threads to be used for training the model.
'''
# Fit the model
my_model_2.fit(X_train, y_train, early_stopping_rounds=5,
             eval_set=[(X_valid, y_valid)],
             verbose=False)
# Get predictions
predictions = my_model_2.predict(X_valid)

# Calculate MAE
mae_2 = mean_absolute_error(predictions, y_valid)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_2)

/usr/local/lib/python3.9/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Mean Absolute Error: 3396.9435905424025


In [ ]:
# Get predictions
predictions_2 = my_model_2.predict(X_test)
result = pd.DataFrame({'PRODUCT_ID': X_test.iloc[:, 0] , 'PRODUCT_LENGTH': predictions_2})
result.to_csv('drive/My Drive/datasetb2d9982/submission.csv')